<h3>1. Web Scraping</h3>

In this first cell we obtain the Html of the Wiki page with a package called Beautiful Soup.

In [3]:
import pandas as pd
import requests
#!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
from lxml import html

link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

res = requests.get(link)

page = BeautifulSoup(res.text, "html.parser")

Next we create the data frame with the labels, we find the rows of the table and add it into an iterator, we skip of one row for exclude the header and last we add the info of each Toronto postal codes with a foreach cicle.

In [24]:
columns = ["PostalCode", "Borough", "Neighborhood"]

df = pd.DataFrame(columns = columns) 

table = page.find("table", {"class": "wikitable"})
rows = table.find_all("tr")
rows = iter(rows) #list iterator
next(rows)

for row in rows:
    data = row.findChildren("td")
    
    df = df.append({
            "PostalCode" : data[0].get_text().replace("\n",""),
            "Borough" : data[1].get_text().replace("\n",""),
            "Neighborhood" : data[2].get_text().replace("\n",""),
        }, ignore_index = True)
    
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


We ignore the cells with a borough that is 'Not assigned'.
If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough. 

In [32]:
df = df[df.Borough != "Not assigned"]

df["Neighborhood"].loc[df.Neighborhood == "Not assigned"] = df["Borough"]

df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


With more than one neighborhood that have the same postal code area, we join it into one row with the neighborhoods separated with a comma.

In [37]:
dataFrame = df.groupby(["PostalCode","Borough"],as_index=False).agg(lambda x: ','.join(x))

dataFrame.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


We print the number of rows of the dataframe.

In [16]:
df.shape

(211, 3)